## Data Preprocessing

In [2]:
import pandas as pd

df_train = pd.read_csv("data/Train.csv")
df_test = pd.read_csv('data/Test.csv')

df_train


,ID,DATOP,FLTID,DEPSTN,ARRSTN,STD,STA,STATUS,AC,target
0,train_id_0,2016-01-03,TU 0712,CMN,TUN,2016-01-03 10:30:00,2016-01-03 12.55.00,ATA,TU 32AIMN,260.0
1,train_id_1,2016-01-13,TU 0757,MXP,TUN,2016-01-13 15:05:00,2016-01-13 16.55.00,ATA,TU 31BIMO,20.0
2,train_id_2,2016-01-16,TU 0214,TUN,IST,2016-01-16 04:10:00,2016-01-16 06.45.00,ATA,TU 32AIMN,0.0
3,train_id_3,2016-01-17,TU 0480,DJE,NTE,2016-01-17 14:10:00,2016-01-17 17.00.00,ATA,TU 736IOK,0.0
4,train_id_4,2016-01-17,TU 0338,TUN,ALG,2016-01-17 14:30:00,2016-01-17 15.50.00,ATA,TU 320IMU,22.0
...,...,...,...,...,...,...,...,...,...,...
107828,train_id_107828,2018-07-05,WKL 0000,TUN,TUN,2018-07-05 23:00:00,2018-07-06 02.00.00,SCH,TU 32AIML,0.0
107829,train_id_107829,2018-01-13,UG 0003,DJE,TUN,2018-01-13 08:00:00,2018-01-13 09.00.00,SCH,UG AT7AT7,0.0
107830,train_id_107830,2018-11-07,SGT 0000,TUN,TUN,2018-11-07 05:00:00,2018-11-07 12.50.00,SCH,TU 736IOK,0.0
107831,train_id_107831,2018-01-23,UG 0010,TUN,DJE,2018-01-23 18:00:00,2018-01-23 18.45.00,ATA,TU CR9ISA,0.0


## Geodata

In [3]:
from geopy import merge_geodata
from geopy import get_geodata

# add geodata to dataframe
geo_data_train = get_geodata(df_train.DEPSTN, df_train.ARRSTN)
geo_data_test = get_geodata(df_test.DEPSTN, df_test.ARRSTN)

df_train_geo = merge_geodata(df_train, geo_data_train, 'DEPSTN')
df_train_geo = merge_geodata(df_train_geo, geo_data_train, 'ARRSTN')

df_test_geo = merge_geodata(df_test, geo_data_test, 'DEPSTN')
df_test_geo = merge_geodata(df_test_geo, geo_data_test, 'ARRSTN')

## Date data type convertion

In [8]:
from date_pyler import applier_cwise
from date_pyler import to_date
#columnwise datetype conversion on dataframe 
applier_cwise(df_train_geo, ["STD", "STA"], to_date)
applier_cwise(df_test_geo, ["STD", "STA"], to_date)

['-', '-', ' ', ':', ':']


UnboundLocalError: local variable 'date' referenced before assignment

In [137]:
df_test_geo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 107833 entries, 0 to 107832
Data columns (total 20 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   ID                107833 non-null  object        
 1   DATOP             107833 non-null  object        
 2   FLTID             107833 non-null  object        
 3   DEPSTN            107833 non-null  object        
 4   ARRSTN            107833 non-null  object        
 5   STD               107833 non-null  datetime64[ns]
 6   STA               107833 non-null  datetime64[ns]
 7   STATUS            107833 non-null  object        
 8   AC                107833 non-null  object        
 9   target            107833 non-null  float64       
 10  iata_DEPSTN       107501 non-null  object        
 11  country_DEPSTN    107501 non-null  object        
 12  lat_DEPSTN        107501 non-null  float64       
 13  lon_DEPSTN        107501 non-null  float64       
 14  elev

In [26]:
df_train_conv = date_type_converter(df_train_geo, ['STD'], format= '%Y-%m-%d %H:%M:%S')
df_train_conv = date_type_converter(df_train_geo, ['STA'], format= '%Y-%m-%d %H.%M.%S')

df_test_conv = date_type_converter(df_test_geo, ['STD'], format = '%Y-%m-%d %H:%M:%S')
df_test_conv = date_type_converter(df_test_geo, ['STA'], format = '%Y-%m-%d %H.%M.%S')

In [30]:
df_train[['STD', 'STA']].str.findall('\D')

AttributeError: 'DataFrame' object has no attribute 'str'